In [2]:
from catalyst.metrics import auc, dice, iou, precision, recall
from PIL import Image
import cv2
from skimage.io import imread
import os
import numpy as np
import torch
import re
import albumentations as A

In [1]:
pred_dir = '../outputs'
mask_dir = '../data/raw/2. All Segmentation Groundtruths/b. Testing Set'

## EX_EP5

In [3]:
ex_pred = pred_dir + '/EX_lr0.001_ep5'
ex_mask = mask_dir + '/3. Hard Exudates'

In [5]:
num_masks = len(os.listdir(ex_mask))
num_preds = len(os.listdir(ex_pred))

list_preds = [os.path.join(ex_pred, f) for f in os.listdir(ex_pred)]
list_masks = [os.path.join(ex_mask, f) for f in os.listdir(ex_mask)]

example = Image.open(list_masks[0])
example_pred = Image.open(list_preds[0])

w, h = example.size

preds = torch.zeros((num_preds, h, w), dtype=torch.int8)
print(preds.shape)
masks = torch.zeros((num_preds, h, w), dtype=torch.int8)
print(masks.shape)

for i in range(num_preds):
    mask_name = re.sub('.jpg', '_EX.tif', list_preds[i].split('/')[-1])

    if mask_name not in os.listdir(ex_mask) or mask_name == 'IDRiD_56_EX.tif':
        new_mask = np.zeros((h, w))
    else:
        mask = Image.open(list_masks[i]) #size w, h
        new_mask = np.array(mask).astype(np.uint8) #shape h, w
    
    pred = Image.open(list_preds[i])
    pred = np.array(pred).astype(np.uint8)
    pred = (pred > 100).astype(np.uint8)
    resize = A.Resize(h, w)
    pred = A.Compose([resize])(image = pred)['image']

    pred = torch.tensor(pred, dtype=torch.uint8)
    new_mask = torch.tensor(new_mask, dtype=torch.uint8)

    preds[i] = pred
    masks[i] = new_mask

print(masks.unique())
print(preds.unique())

print(f'Dice {dice(preds[:, None, :, :], masks[:, None, :, :]).mean()}')
print(f'IoU {iou(preds[:, None, :, :], masks[:, None, :, :]).mean()}')




torch.Size([27, 2848, 4288])
torch.Size([27, 2848, 4288])
tensor([0, 1], dtype=torch.int8)
tensor([0, 1], dtype=torch.int8)
Dice 0.06675796955823898
IoU 0.0345316156744957
